# 🧾 View as a summary

In [ ]:
#| default_exp repr_str

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# |hide
import os
from nbdev.showdoc import *
from fastcore.test import test_eq, test

In [ ]:
# |hide
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [ ]:
#| hide
#| export

import warnings
from typing import Union, Optional as O

import numpy as np
import jax, jax.numpy as jnp

from lovely_numpy import np_to_str_common, pretty_str, sparse_join, ansi_color, in_debugger
from lovely_numpy import config as lnp_config

from lovely_jax.utils.config import get_config
from lovely_jax.utils.misc import to_numpy, is_cpu, test_array_repr

In [ ]:
# |hide
# torch.manual_seed(42)


key = jax.random.PRNGKey(0)
randoms = jax.random.normal(key, (100,))

In [ ]:
spicy = (randoms[:12].at[0].mul(10000)
                    .at[1].divide(10000)
                    .at[3].set(float('inf'))
                    .at[4].set(float('-inf'))
                    .at[5].set(float('nan'))
                    .reshape((2,6)))

In [ ]:
# |exporti
dtnames =   {   "float16": "f16",
                "float32": "", # Default dtype in jax
                "float64": "f64", 
                "bfloat16": "bf16",
                "uint8": "u8",
                "uint16": "u16",
                "uint32": "u32",
                "uint64": "u64",
                "int8": "i8",
                "int16": "i16",
                "int32": "i32",
                "int64": "i64",
            }

def short_dtype(x: jax.Array) -> str:
    return dtnames.get(x.dtype.name, str(x.dtype))

In [ ]:
# |hide
test_eq(short_dtype(jnp.array(1., dtype=jnp.bfloat16)), "bf16")

In [ ]:
# | exporti
def plain_repr(x: jax.Array):
    "Pick the right function to get a plain repr"
    # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}" # Could be a sub-class.
    return x._plain_repr() if hasattr(type(x), "_plain_repr") else repr(x)

# def plain_str(x: torch.Tensor):
#     "Pick the right function to get a plain str."
#     # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}"
#     return x._plain_str() if hasattr(type(x), "_plain_str") else str(x)

In [ ]:
# | exporti
def is_nasty(x: jax.Array):
    """Return true of any `x` values are inf or nan"""
    
    if x.size == 0: return False # min/max don't like zero-lenght arrays
    
    x_min = x.min()
    x_max = x.max()
    
    return jnp.isnan(x_min) or jnp.isinf(x_min) or jnp.isinf(x_max)

In [ ]:
#| hide

test_eq(is_nasty(jnp.array([1, 2, float("nan")])), True)
test_eq(is_nasty(jnp.array([1, 2, float("inf")])), True)
test_eq(is_nasty(jnp.array([1, 2, 3])), False)
test_eq(is_nasty(jnp.array([])), False)

In [ ]:
# |export
def jax_to_str_common(x: jax.Array,  # Input
                        color=True,                     # ANSI color highlighting
                        ddof=0):                        # For "std" unbiasing

    if x.size == 0:
        return ansi_color("empty", "grey", color)

    zeros = ansi_color("all_zeros", "grey", color) if jnp.equal(x, 0.).all() and x.size > 1 else None
    pinf = ansi_color("+Inf!", "red", color) if jnp.isposinf(x).any() else None
    ninf = ansi_color("-Inf!", "red", color) if jnp.isneginf(x).any() else None
    nan = ansi_color("NaN!", "red", color) if jnp.isnan(x).any() else None

    attention = sparse_join([zeros,pinf,ninf,nan])
    numel = f"n={x.size}" if x.size > 5 and max(x.shape) != x.size else None

    summary = None
    if not zeros and x.ndim > 0:
        # Calculate stats on good values only.
        # This is memory expensive, don't do it on GPU.
        # We divert to numpy if there are any nasties in the data.
        # gx = x[ jnp.isfinite(x) ]

        minmax = f"x∈[{pretty_str(x.min())}, {pretty_str(x.max())}]" if x.size > 2 else None
        meanstd = f"μ={pretty_str(x.mean())} σ={pretty_str(x.std(ddof=ddof))}" if x.size >= 2 else None
        summary = sparse_join([numel, minmax, meanstd])


    return sparse_join([ summary, attention])

In [ ]:
# |exporti

def to_str(x: jax.Array,  # Input
            plain: bool=False,
            verbose: bool=False,
            depth=0,
            lvl=0,
            color=None) -> str:

    if plain:
        return plain_repr(x)

    conf = get_config()

    tname = type(x).__name__.split(".")[-1]
    shape = str(list(x.shape)) if x.ndim else None
    type_str = sparse_join([tname, shape], sep="")
    

    dev = f"{x.device().platform}:{x.device().id}"
    dtype = short_dtype(x)
    # grad_fn = t.grad_fn.name() if t.grad_fn else None
    # PyTorch does not want you to know, but all `grad_fn``
    # tensors actuall have `requires_grad=True`` too.
    # grad = "grad" if t.requires_grad else None 
    grad = grad_fn = None


    # For complex tensors, just show the shape / size part for now.
    if not jnp.iscomplexobj(x):
        if color is None: color=conf.color
        if in_debugger(): color = False
        # `lovely-numpy` is used to calculate stats when doing so on GPU would require
        # memory allocation (not float tensors, tensors with bad numbers), or if the
        # data is on CPU (because numpy is faster).
        #
        # Temporarily set the numpy config to match our config for consistency.
        with lnp_config(precision=conf.precision,
                        threshold_min=conf.threshold_min,
                        threshold_max=conf.threshold_max,
                        sci_mode=conf.sci_mode):

            if is_cpu(x) or is_nasty(x):
                common = np_to_str_common(np.array(x), color=color)
            else:
                common = jax_to_str_common(x, color=color)

            vals = pretty_str(x) if 0 < x.size <= 10 else None
            res = sparse_join([type_str, dtype, common, grad, grad_fn, dev, vals])
    else:
        res = plain_repr(x)


    if verbose:
        res += "\n" + plain_repr(x)

    if depth and x.ndim > 1:

        deep_width = min((x.shape[0]), conf.deeper_width) # Print at most this many lines
        deep_lines = [ " "*conf.indent*(lvl+1) + to_str(x[i,:], depth=depth-1, lvl=lvl+1)
                            for i in range(deep_width)] 

        # If we were limited by width, print ...
        if deep_width < x.shape[0]: deep_lines.append(" "*conf.indent*(lvl+1) + "...")

        res += "\n" + "\n".join(deep_lines)

    return res

In [ ]:
# |exporti
def history_warning():
    "Issue a warning (once) ifw e are running in IPYthon with output cache enabled"

    if "get_ipython" in globals() and get_ipython().cache_size > 0:
        warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")

In [ ]:
# |hide
get_ipython().cache_size=1000
history_warning()

/tmp/ipykernel_662013/3648473780.py:6: UserWarning: IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html
  warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")


In [ ]:
# |hide
get_ipython().cache_size=0

In [ ]:
#| exporti

class StrProxy():
    def __init__(self, x: jax.Array, plain=False, verbose=False, depth=0, lvl=0, color=None):
        self.x = x
        self.plain = plain
        self.verbose = verbose
        self.depth=depth
        self.lvl=lvl
        self.color=color
        history_warning()
    
    def __repr__(self):
        return to_str(self.x, plain=self.plain, verbose=self.verbose,
                      depth=self.depth, lvl=self.lvl, color=self.color)

    # This is used for .deeper attribute and .deeper(depth=...).
    # The second onthe results in a __call__.
    def __call__(self, depth=1):
        return StrProxy(self.x, depth=depth)

In [ ]:
# |export
def lovely(x: jax.Array, # Tensor of interest
            verbose=False,  # Whether to show the full tensor
            plain=False,    # Just print if exactly as before
            depth=0,        # Show stats in depth
            color=None):    # Force color (True/False) or auto.
    return StrProxy(x, verbose=verbose, plain=plain, depth=depth, color=color)

### Examples

In [ ]:
print(lovely(randoms[0]))
print(lovely(randoms[:2]))
print(lovely(randoms[:6].reshape((2, 3)))) # More than 2 elements -> show statistics
print(lovely(randoms[:11]))           # More than 10 -> suppress data output


Array gpu:0 -1.981
Array[2] μ=-0.466 σ=1.515 gpu:0 [-1.981, 1.048]
Array[2, 3] n=6 x∈[-1.981, 1.048] μ=-0.017 σ=1.113 gpu:0 [[-1.981, 1.048, 0.890], [0.035, -0.947, 0.851]]
Array[11] x∈[-1.981, 1.048] μ=-0.191 σ=0.899 gpu:0


In [ ]:
# |hide
test_array_repr(str(lovely(randoms[0])),                "Array gpu:0 -1.981")
test_array_repr(str(lovely(randoms[:2])),               "Array[2] μ=-0.466 σ=1.515 gpu:0 [-1.981, 1.048]")
test_array_repr(str(lovely(randoms[:6].reshape(2, 3))), "Array[2, 3] n=6 x∈[-1.981, 1.048] μ=-0.017 σ=1.113 gpu:0 [[-1.981, 1.048, 0.890], [0.035, -0.947, 0.851]]")
test_array_repr(str(lovely(randoms[:11])),              "Array[11] x∈[-1.981, 1.048] μ=-0.191 σ=0.899 gpu:0")

In [ ]:
grad = jnp.array(1., dtype=jnp.float16)
print(lovely(grad)); print(lovely(grad+1))

Array f16 gpu:0 1.000
Array f16 gpu:0 2.000


In [ ]:
# |hide
# test_eq(str(lovely(grad)), "tensor f64 grad 1.000")
# test_eq(str(lovely(grad+1)), "tensor f64 grad AddBackward0 2.000")

In [ ]:
# if torch.cuda.is_available():
#     print(lovely(torch.tensor(1., device=torch.device("cuda:0"))))
#     test_eq(str(lovely(torch.tensor(1., device=torch.device("cuda:0")))), "tensor cuda:0 1.000")

Do we have __any__ floating point nasties? Is the tensor __all__ zeros?

In [ ]:
# Statistics and range are calculated on good values only, if there are at lest 3 of them.
lovely(spicy)

Array[2, 6] n=12 x∈[-1.981e+04, 0.890] μ=-2.201e+03 σ=6.226e+03 +Inf! -Inf! NaN! gpu:0

In [ ]:
# |hide
test_array_repr(str(lovely(spicy)),
    'Array[2, 6] n=12 x∈[-1.981e+04, 0.890] μ=-2.201e+03 σ=6.226e+03 \x1b[31m+Inf!\x1b[0m \x1b[31m-Inf!\x1b[0m \x1b[31mNaN!\x1b[0m gpu:0')

In [ ]:
lovely(spicy, color=False)

Array[2, 6] n=12 x∈[-1.981e+04, 0.890] μ=-2.201e+03 σ=6.226e+03 +Inf! -Inf! NaN! gpu:0

In [ ]:
str(lovely(jnp.array([float("nan")]*11)))

'Array[11] \x1bNaN!\x1b gpu:0'

In [ ]:
# |hide 
test_array_repr(str(lovely(jnp.array([float("nan")]*11))),
        'Array[11] \x1b[31mNaN!\x1b[0m gpu:0')

In [ ]:
lovely(jnp.zeros(12))

Array[12] all_zeros gpu:0

In [ ]:
# |hide
test_array_repr(str(lovely(jnp.zeros(12))),
        'Array[12] \x1b[38;2;127;127;127mall_zeros\x1b[0m gpu:0')

In [ ]:
lovely(jnp.array([], dtype=jnp.float16).reshape((0,0,0)))

Array[0, 0, 0] f16 empty gpu:0

In [ ]:
# |hide
test_array_repr(str(lovely(jnp.array([], dtype=jnp.float16).reshape((0,0,0)))),
        'Array[0, 0, 0] f16 \x1b[38;2;127;127;127mempty\x1b[0m gpu:0')

In [ ]:
lovely(jnp.array([1,2,3], dtype=jnp.int32))

Array[3] i32 x∈[1, 3] μ=2.000 σ=0.816 gpu:0 [1, 2, 3]

In [ ]:
# |hide
test_array_repr(str(lovely(jnp.array([1,2,3], dtype=jnp.int32))),
        'Array[3] i32 x∈[1, 3] μ=2.000 σ=0.816 gpu:0 [1, 2, 3]')

In [ ]:
jnp.set_printoptions(linewidth=120, precision=2)
lovely(spicy, verbose=True)

Array[2, 6] n=12 x∈[-1.981e+04, 0.890] μ=-2.201e+03 σ=6.226e+03 +Inf! -Inf! NaN! gpu:0
Array([[-1.98e+04,  1.05e-04,  8.90e-01,       inf,      -inf,       nan],
       [ 3.12e-02, -3.90e-01,  1.32e-02, -4.21e-01, -1.23e+00, -1.25e+00]], dtype=float32)

In [ ]:
lovely(spicy, plain=True)

Array([[-1.98e+04,  1.05e-04,  8.90e-01,       inf,      -inf,       nan],
       [ 3.12e-02, -3.90e-01,  1.32e-02, -4.21e-01, -1.23e+00, -1.25e+00]], dtype=float32)

In [ ]:
image = jnp.load("mysteryman.npy")
image = image.at[1,2,3].set(float('nan'))

lovely(image, depth=2) # Limited by set_config(deeper_lines=N)

Array[3, 196, 196] n=115248 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 NaN! gpu:0
  Array[196, 196] n=38416 x∈[-2.118, 2.249] μ=-0.324 σ=1.036 gpu:0
    Array[196] x∈[-1.912, 2.249] μ=-0.673 σ=0.521 gpu:0
    Array[196] x∈[-1.861, 2.163] μ=-0.738 σ=0.417 gpu:0
    Array[196] x∈[-1.758, 2.198] μ=-0.806 σ=0.396 gpu:0
    Array[196] x∈[-1.656, 2.249] μ=-0.849 σ=0.368 gpu:0
    Array[196] x∈[-1.673, 2.198] μ=-0.857 σ=0.356 gpu:0
    Array[196] x∈[-1.656, 2.146] μ=-0.848 σ=0.371 gpu:0
    Array[196] x∈[-1.433, 2.215] μ=-0.784 σ=0.396 gpu:0
    Array[196] x∈[-1.279, 2.249] μ=-0.695 σ=0.485 gpu:0
    Array[196] x∈[-1.364, 2.249] μ=-0.637 σ=0.538 gpu:0
    ...
  Array[196, 196] n=38416 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 NaN! gpu:0
    Array[196] x∈[-1.861, 2.411] μ=-0.529 σ=0.555 gpu:0
    Array[196] x∈[-1.826, 2.359] μ=-0.562 σ=0.472 gpu:0
    Array[196] x∈[-1.756, 2.376] μ=-0.622 σ=0.458 NaN! gpu:0
    Array[196] x∈[-1.633, 2.429] μ=-0.664 σ=0.429 gpu:0
    Array[196] x∈[-1.651, 2.376] μ=-0.669 σ=0.

In [ ]:
# |hide
#### CUDA memory is not leaked

In [ ]:
# |hide
# |eval: false
# def memstats():
#     allocated = int(torch.cuda.memory_allocated() // (1024*1024))
#     max_allocated = int(torch.cuda.max_memory_allocated() // (1024*1024))
#     return f"Allocated: {allocated} MB, Max: {max_allocated} Mb"

# if torch.cuda.is_available():
#     cudamem = torch.cuda.memory_allocated()
#     print(f"before allocation: {memstats()}")
#     numbers = torch.randn((3, 1024, 1024), device="cuda") # 12Mb image
#     torch.cuda.synchronize()

#     print(f"after allocation: {memstats()}")
#     # Note, the return value of lovely() is not a string, but a
#     # StrProxy that holds reference to 'numbers'. You have to del
#     # the references to it, but once it's gone, the reference to
#     # the tensor is gone too.
#     display(lovely(numbers) )
#     print(f"after repr: {memstats()}")
    
#     del numbers
#     # torch.cuda.memory.empty_cache()

#     print(f"after cleanup: {memstats()}")
#     test_eq(cudamem >= torch.cuda.memory_allocated(), True)

In [ ]:
# We don't really supposed complex numbers yet
c = jnp.array([-0.4011-0.4035j,  1.1300+0.0788j, -0.0277+0.9978j, -0.4636+0.6064j, -1.1505-0.9865j])
lovely(c)

Array([-0.4 -0.4j ,  1.13+0.08j, -0.03+1.j  , -0.46+0.61j, -1.15-0.99j], dtype=complex64)